# TensorFlow类型转换与类型冲突

TensorFlow采用严格的类型检查机制，不同于NumPy的自动类型提升。本notebook详细讲解类型系统的设计原理、常见冲突场景及解决方案。

## 学习目标
1. 理解TensorFlow严格类型检查的设计理念
2. 识别常见的类型冲突错误
3. 掌握tf.cast等类型转换方法
4. 了解类型转换的性能影响

## 1. 环境设置

In [ ]:
import tensorflow as tf
import numpy as np

# 设置随机种子
RANDOM_SEED = 42
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

## 2. TensorFlow的严格类型系统

### 2.1 设计理念

TensorFlow之所以采用严格类型检查，主要基于以下考虑：

1. **性能优化**: GPU和TPU对特定数据类型有硬件级优化
2. **避免隐式错误**: 自动类型转换可能导致精度损失而不被察觉
3. **内存效率**: 明确的类型控制有助于优化内存使用
4. **计算图优化**: 静态类型使编译器能更好地优化计算图

In [ ]:
# NumPy vs TensorFlow 类型处理对比

print("=" * 60)
print("NumPy: 自动类型提升 (Type Promotion)")
print("=" * 60)

# NumPy会自动将int转换为float
np_int = np.array([1, 2, 3])
np_float = np.array([1.5, 2.5, 3.5])
np_result = np_int + np_float
print(f"int数组: {np_int.dtype}")
print(f"float数组: {np_float.dtype}")
print(f"相加结果类型: {np_result.dtype}")
print(f"相加结果: {np_result}\n")

print("=" * 60)
print("TensorFlow: 严格类型检查")
print("=" * 60)

tf_int = tf.constant([1, 2, 3])
tf_float = tf.constant([1.5, 2.5, 3.5])
print(f"int张量: {tf_int.dtype}")
print(f"float张量: {tf_float.dtype}")
print("尝试相加会触发类型错误...")

## 3. 常见类型冲突场景

### 3.1 整数与浮点数运算

In [ ]:
# 场景1: 整数与浮点数运算

float_tensor = tf.constant(2.0)   # float32
int_tensor = tf.constant(50)       # int32

print(f"float_tensor dtype: {float_tensor.dtype}")
print(f"int_tensor dtype: {int_tensor.dtype}\n")

# 尝试相加（会报错）
try:
    result = float_tensor + int_tensor
    print(f"相加结果: {result}")
except tf.errors.InvalidArgumentError as e:
    print(f"类型错误: 无法将float32与int32相加")
    print(f"错误信息: cannot compute AddV2 as input types don't match")

In [ ]:
# 解决方案: 使用tf.cast进行类型转换

float_tensor = tf.constant(2.0)
int_tensor = tf.constant(50)

# 方案1: 将整数转换为浮点数（推荐，避免精度损失）
int_as_float = tf.cast(int_tensor, dtype=tf.float32)
result1 = float_tensor + int_as_float
print(f"方案1 - int转float: {result1}")

# 方案2: 将浮点数转换为整数（可能损失精度）
float_as_int = tf.cast(float_tensor, dtype=tf.int32)
result2 = float_as_int + int_tensor
print(f"方案2 - float转int: {result2}")
print(f"注意: 2.0 -> 2，小数部分被截断")

### 3.2 不同精度的浮点数运算

In [ ]:
# 场景2: float32与float64混合运算

f32_tensor = tf.constant([1.0, 2.0, 3.0], dtype=tf.float32)
f64_tensor = tf.constant([1.0, 2.0, 3.0], dtype=tf.float64)

print(f"float32张量: {f32_tensor.dtype}")
print(f"float64张量: {f64_tensor.dtype}\n")

# 尝试相加
try:
    result = f32_tensor + f64_tensor
except tf.errors.InvalidArgumentError as e:
    print("类型错误: float32和float64不能直接运算")

# 解决方案
f64_to_f32 = tf.cast(f64_tensor, dtype=tf.float32)
result = f32_tensor + f64_to_f32
print(f"\n转换后相加结果: {result}")
print(f"结果类型: {result.dtype}")

### 3.3 布尔类型与数值类型

In [ ]:
# 场景3: 布尔张量与数值张量运算

bool_tensor = tf.constant([True, False, True])
int_tensor = tf.constant([1, 2, 3])

print(f"布尔张量: {bool_tensor}, dtype: {bool_tensor.dtype}")
print(f"整数张量: {int_tensor}, dtype: {int_tensor.dtype}\n")

# 布尔转整数（True->1, False->0）
bool_as_int = tf.cast(bool_tensor, dtype=tf.int32)
print(f"布尔转整数: {bool_as_int}")

# 布尔转浮点数
bool_as_float = tf.cast(bool_tensor, dtype=tf.float32)
print(f"布尔转浮点: {bool_as_float}")

# 应用场景: 掩码操作
values = tf.constant([10.0, 20.0, 30.0])
mask = tf.constant([True, False, True])
masked_values = values * tf.cast(mask, tf.float32)
print(f"\n掩码应用: {masked_values}")

## 4. tf.cast详解

`tf.cast`是TensorFlow中最常用的类型转换函数。

**函数签名:**
```python
tf.cast(x, dtype, name=None)
```

**参数:**
- `x`: 输入张量
- `dtype`: 目标数据类型
- `name`: 操作名称（可选）

In [ ]:
# tf.cast 使用示例

original = tf.constant([1.9, 2.5, 3.1, -1.7])
print(f"原始张量: {original}\n")

# 浮点转整数（截断，非四舍五入）
to_int32 = tf.cast(original, tf.int32)
print(f"转int32: {to_int32} (截断小数)")

# 浮点转无符号整数
positive = tf.constant([1.9, 2.5, 3.1])
to_uint8 = tf.cast(positive, tf.uint8)
print(f"转uint8: {to_uint8}")

# 整数转浮点
int_tensor = tf.constant([1, 2, 3, 4])
to_float32 = tf.cast(int_tensor, tf.float32)
print(f"int转float32: {to_float32}")

# 降低精度
f64 = tf.constant([1.123456789012345, 2.987654321098765], dtype=tf.float64)
f32 = tf.cast(f64, tf.float32)
print(f"\nfloat64: {f64}")
print(f"float32: {f32} (精度损失)")

### 4.1 浮点转整数的舍入方式

In [ ]:
# 浮点数转整数的不同舍入方式

values = tf.constant([1.2, 1.5, 1.7, 2.5, -1.2, -1.5, -1.7])
print(f"原始值: {values.numpy()}\n")

# tf.cast - 直接截断（向零取整）
truncated = tf.cast(values, tf.int32)
print(f"tf.cast (截断): {truncated.numpy()}")

# tf.math.floor - 向下取整
floored = tf.cast(tf.math.floor(values), tf.int32)
print(f"floor (向下取整): {floored.numpy()}")

# tf.math.ceil - 向上取整
ceiled = tf.cast(tf.math.ceil(values), tf.int32)
print(f"ceil (向上取整): {ceiled.numpy()}")

# tf.math.round - 四舍五入（银行家舍入法）
rounded = tf.cast(tf.math.round(values), tf.int32)
print(f"round (四舍五入): {rounded.numpy()}")
print("注意: round使用银行家舍入，0.5向最近的偶数取整")

## 5. TensorFlow常用数据类型

In [ ]:
# TensorFlow常用数据类型一览

dtype_info = [
    ("tf.float16", "半精度浮点", "16位，GPU加速，精度较低"),
    ("tf.float32", "单精度浮点", "32位，默认浮点类型，最常用"),
    ("tf.float64", "双精度浮点", "64位，高精度计算"),
    ("tf.int8", "8位整数", "量化模型常用"),
    ("tf.int16", "16位整数", "较少使用"),
    ("tf.int32", "32位整数", "默认整数类型"),
    ("tf.int64", "64位整数", "大索引值"),
    ("tf.uint8", "无符号8位", "图像数据(0-255)"),
    ("tf.bool", "布尔类型", "逻辑运算、掩码"),
    ("tf.string", "字符串", "文本处理"),
    ("tf.complex64", "复数", "信号处理"),
]

print(f"{'类型':<15} {'名称':<12} {'说明'}")
print("=" * 60)
for dtype, name, desc in dtype_info:
    print(f"{dtype:<15} {name:<12} {desc}")

## 6. 类型转换的性能影响

In [ ]:
import time

# 类型转换性能测试

# 创建大型张量
large_f32 = tf.random.normal((10000, 10000), dtype=tf.float32)
n_iterations = 100

# 测试类型转换开销
start = time.time()
for _ in range(n_iterations):
    _ = tf.cast(large_f32, tf.float64)
cast_time = time.time() - start
print(f"float32->float64 ({n_iterations}次): {cast_time:.4f}秒")

# 测试不进行类型转换的运算
start = time.time()
for _ in range(n_iterations):
    _ = large_f32 + large_f32
no_cast_time = time.time() - start
print(f"同类型运算 ({n_iterations}次): {no_cast_time:.4f}秒")

print(f"\n类型转换带来的额外开销: {(cast_time/no_cast_time - 1)*100:.1f}%")

## 7. 最佳实践

In [ ]:
# 最佳实践示例

# 1. 在模型输入时统一数据类型
def prepare_data(data):
    """确保输入数据为float32类型"""
    if isinstance(data, np.ndarray):
        data = data.astype(np.float32)
    return tf.convert_to_tensor(data, dtype=tf.float32)

# 2. 在计算开始前进行必要的类型转换
def safe_divide(a, b):
    """类型安全的除法操作"""
    a = tf.cast(a, tf.float32)
    b = tf.cast(b, tf.float32)
    return a / b

# 3. 使用断言确保类型正确
def type_checked_operation(tensor):
    """带类型检查的操作"""
    tf.debugging.assert_type(tensor, tf.float32, 
                             message="输入必须是float32类型")
    return tf.nn.relu(tensor)

# 演示
test_data = np.array([1, 2, 3, 4, 5])
prepared = prepare_data(test_data)
print(f"准备后的数据类型: {prepared.dtype}")

result = safe_divide(tf.constant(10), tf.constant(3))
print(f"安全除法结果: {result}")

## 知识点总结

### 类型冲突解决方案一览

| 冲突场景 | 解决方法 | 注意事项 |
|---------|---------|----------|
| int + float | `tf.cast(int_tensor, tf.float32)` | 优先将int转float |
| float32 + float64 | `tf.cast(f64, tf.float32)` | 选择目标精度 |
| bool + numeric | `tf.cast(bool_tensor, dtype)` | 用于掩码操作 |
| float -> int | `tf.cast(tensor, tf.int32)` | 截断而非四舍五入 |

### 关键要点

1. **TensorFlow不进行自动类型转换** - 这是设计决策，非缺陷
2. **tf.cast是核心转换函数** - 几乎所有类型转换都通过它完成
3. **注意精度损失** - float64→float32、float→int都会损失精度
4. **类型转换有性能开销** - 尽量在数据准备阶段统一类型
5. **float32是深度学习的首选** - GPU优化的主要目标类型